<a href="https://colab.research.google.com/github/Pitypety/tweet_binary_classification/blob/main/binary_classification_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tweet-ek osztályzása zenei műfaj alapján (jazz és rock)


#Előkészületek, fájlbeolvasás google drive-ból

In [36]:
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk import pos_tag
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import re
import string

In [37]:
#Fájlbeolvasás githubról
url_jazz = 'https://raw.githubusercontent.com/Pitypety/tweet_binary_classification/main/jazz7.csv'
url_rock = 'https://raw.githubusercontent.com/Pitypety/tweet_binary_classification/main/rock7.csv'
df_jazz = pd.read_csv(url_jazz)
df_rock = pd.read_csv(url_rock)
df_jazz

,tweet_text,all_hashtags
0,"Now Playing, Lynne Arriale — Think of One from...","['music', 'Jazz']"
1,Live #MagaSwing #jazz #musique #culture Regard...,"['MagaSwing', 'jazz', 'musique', 'culture']"
2,Subscribe to our playlist !⠀ ⠀ https://t.co/Bj...,"['playlist', 'jazz', 'newyork', 'bar', 'wnts',..."
3,"Now Playing, Kenny Burrell — The Squeeze from ...","['music', 'Jazz']"
4,Two Left Feet - 12/02/21 Pre save from midnig...,"['newmusic', 'bassist', 'bass', 'guitar', 'gui..."
...,...,...
7631,Afro Brazilian Jazz Journey https://t.co/ZgWR...,"['jazz', 'drums', 'drurmmer', 'live', 'show', ..."
7632,* * I've noticed that the more adventurous a...,"['jazz', 'art', 'quote']"
7633,"Now Playing, Dexter Gordon — Lullaby In Rhythm...","['music', 'Jazz']"
7634,10 games ahead of us tonight 🙌🔥#NBA2021 Bet Ea...,"['NBA2021', 'Warriors', 'Jazz', 'Nets', 'Rapto..."


#Posztok címkézése, tanító és teszt adatbázisok elkülönítése

In [38]:
#dataframe-ek felcímkézése
df_jazz['Label']='jazz' 
df_rock['Label']='rock'
df_jazz

,tweet_text,all_hashtags,Label
0,"Now Playing, Lynne Arriale — Think of One from...","['music', 'Jazz']",jazz
1,Live #MagaSwing #jazz #musique #culture Regard...,"['MagaSwing', 'jazz', 'musique', 'culture']",jazz
2,Subscribe to our playlist !⠀ ⠀ https://t.co/Bj...,"['playlist', 'jazz', 'newyork', 'bar', 'wnts',...",jazz
3,"Now Playing, Kenny Burrell — The Squeeze from ...","['music', 'Jazz']",jazz
4,Two Left Feet - 12/02/21 Pre save from midnig...,"['newmusic', 'bassist', 'bass', 'guitar', 'gui...",jazz
...,...,...,...
7631,Afro Brazilian Jazz Journey https://t.co/ZgWR...,"['jazz', 'drums', 'drurmmer', 'live', 'show', ...",jazz
7632,* * I've noticed that the more adventurous a...,"['jazz', 'art', 'quote']",jazz
7633,"Now Playing, Dexter Gordon — Lullaby In Rhythm...","['music', 'Jazz']",jazz
7634,10 games ahead of us tonight 🙌🔥#NBA2021 Bet Ea...,"['NBA2021', 'Warriors', 'Jazz', 'Nets', 'Rapto...",jazz


In [39]:
#szétvágás tanító és teszt adatbázisra
learn = [df_jazz[0:5340], df_rock[0:5340]]
test = [df_rock[5341:7637],df_jazz[5341:7637]]

In [40]:
#jazz és rock dataframek egyesítése külön tanító és teszt adatbázisokba
train_dataframe = pd.concat(learn, ignore_index=True)
test_dataframe = pd.concat(test, ignore_index=True)

# Adatbázisok szűrése, tisztítása

In [41]:
#Url-ek eltávolítása tanító adatbázisból
train_dataframe['tweet_text'] = train_dataframe['tweet_text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])

In [42]:
#Url-ek eltávolítása teszt adatbázisból
test_dataframe['tweet_text'] = test_dataframe['tweet_text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])

#NLP

In [43]:
#Lowercase Sample
text = "It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife."
text = text.lower()
print(text[:100])

it is a truth universally acknowledged, that a single man in possession of a good fortune, must be i


In [44]:
#Lowercase for Dataframe
text = ""
for x in train_dataframe['tweet_text']:
  text += x.lower()

print(text[:200])

now playing, lynne arriale — think of one from live at the montreux jazz festival #music #jazzlive #magaswing #jazz #musique #culture regardez megaswing quintet live subscribe to our playlist !⠀ ⠀ now


In [45]:
#removing punctuation Sample
#print(string.punctuation)
text_p = "".join([char for char in text if char not in string.punctuation])
print(text_p[:200])

now playing lynne arriale — think of one from live at the montreux jazz festival music jazzlive magaswing jazz musique culture regardez megaswing quintet live subscribe to our playlist ⠀ ⠀ now playing


In [46]:
#word tokenization Sample
import nltk
nltk.download('punkt')
words = word_tokenize(text_p)
print(words[:50])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['now', 'playing', 'lynne', 'arriale', '—', 'think', 'of', 'one', 'from', 'live', 'at', 'the', 'montreux', 'jazz', 'festival', 'music', 'jazzlive', 'magaswing', 'jazz', 'musique', 'culture', 'regardez', 'megaswing', 'quintet', 'live', 'subscribe', 'to', 'our', 'playlist', '⠀', '⠀', 'now', 'playing', 'kenny', 'burrell', '—', 'the', 'squeeze', 'from', 'bluesin', 'around', 'music', 'jazztwo', 'left', 'feet', '120221', 'pre', 'save', 'from', 'midnight']


In [47]:
#stopword filtering
nltk.download('stopwords')
stop_words = stopwords.words('english')
print(stop_words[:50])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be']


In [48]:
filtered_words = [word for word in words if word not in stop_words]
print(filtered_words[:50])

['playing', 'lynne', 'arriale', '—', 'think', 'one', 'live', 'montreux', 'jazz', 'festival', 'music', 'jazzlive', 'magaswing', 'jazz', 'musique', 'culture', 'regardez', 'megaswing', 'quintet', 'live', 'subscribe', 'playlist', '⠀', '⠀', 'playing', 'kenny', 'burrell', '—', 'squeeze', 'bluesin', 'around', 'music', 'jazztwo', 'left', 'feet', '120221', 'pre', 'save', 'midnight', 'tonight', 'link', 'bio', 'newmusic', 'bassist', 'bass', 'guitar', 'guitarist', 'music', 'funk', 'jazz']


In [49]:
#stemming
porter = PorterStemmer()
stemmed = [porter.stem(word) for word in filtered_words]
print(stemmed[:50])

['play', 'lynn', 'arrial', '—', 'think', 'one', 'live', 'montreux', 'jazz', 'festiv', 'music', 'jazzliv', 'magasw', 'jazz', 'musiqu', 'cultur', 'regardez', 'megasw', 'quintet', 'live', 'subscrib', 'playlist', '⠀', '⠀', 'play', 'kenni', 'burrel', '—', 'squeez', 'bluesin', 'around', 'music', 'jazztwo', 'left', 'feet', '120221', 'pre', 'save', 'midnight', 'tonight', 'link', 'bio', 'newmus', 'bassist', 'bass', 'guitar', 'guitarist', 'music', 'funk', 'jazz']


In [50]:
#POS sample
nltk.download('averaged_perceptron_tagger')
pos = pos_tag(filtered_words)
print(pos[:50])

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[('playing', 'VBG'), ('lynne', 'JJ'), ('arriale', 'JJ'), ('—', 'NN'), ('think', 'VBP'), ('one', 'CD'), ('live', 'JJ'), ('montreux', 'NN'), ('jazz', 'NN'), ('festival', 'NN'), ('music', 'NN'), ('jazzlive', 'NN'), ('magaswing', 'VBG'), ('jazz', 'NN'), ('musique', 'JJ'), ('culture', 'NN'), ('regardez', 'NN'), ('megaswing', 'VBG'), ('quintet', 'JJ'), ('live', 'JJ'), ('subscribe', 'NN'), ('playlist', 'NN'), ('⠀', 'NNP'), ('⠀', 'NNP'), ('playing', 'VBG'), ('kenny', 'NN'), ('burrell', 'NN'), ('—', 'NNP'), ('squeeze', 'NN'), ('bluesin', 'NN'), ('around', 'IN'), ('music', 'NN'), ('jazztwo', 'NN'), ('left', 'VBD'), ('feet', 'NNS'), ('120221', 'CD'), ('pre', 'NNS'), ('save', 'VBP'), ('midnight', 'VBN'), ('tonight', 'NN'), ('link', 'NN'), ('bio', 'NN'), ('newmusic', 'JJ'), ('bassist', 'NN'), ('bass', 'NN'), ('gui

#Osztályozó

In [50]:
train_data.label.hist()